# Intermediate Neural Network in TensorFlow

In this notebook, we convert our [intermediate-depth neural network](https://github.com/the-deep-learners/TensorFlow-LiveLessons/blob/master/notebooks/intermediate_net_in_keras.ipynb) from Keras to TensorFlow following Aymeric Damien's [MLP Notebook](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/multilayer_perceptron.ipynb) style.

#### Load dependencies

In [1]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

#### Load data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


#### Set neural network hyperparameters

In [3]:
lr = 0.1
epochs = 15
batch_size = 128

#### Set number of neurons for each layer

In [4]:
n_input = 784
n_hidden_1 = 64
n_hidden_2 = 64
n_classes = 10

#### Define placeholders for inputs and labels

In [5]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

#### Design neural network architecture

In [6]:
def network(x, weights, biases):
    
    # hidden layer 1: dense with ReLU activation
    dense_1 = tf.add(tf.matmul(x, weights['d1']), biases['d1'])
    dense_1 = tf.nn.relu(dense_1)
    
    # hidden layer 2: dense with ReLU activation
    dense_2 = tf.add(tf.matmul(dense_1, weights['d2']), biases['d2'])
    dense_2 = tf.nn.relu(dense_2)
    
    # linear output layer (softmax )
    out_layer = tf.add(tf.matmul(dense_2, weights['out']), biases['out'])
    
    return out_layer

#### Define dictionaries for storing weights and biases for each layer, initialized with random values

In [7]:
weight_dict = {
    'd1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'd2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
bias_dict = {
    'd1': tf.Variable(tf.random_normal([n_hidden_1])),
    'd2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

#### Build model

In [8]:
predictions = network(x, weights=weight_dict, biases=bias_dict)

#### Define model loss and optimizer

In [9]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(cost)

#### Initialize variables

In [10]:
initializer_op = tf.global_variables_initializer()

#### Train the network in a session

In [11]:
with tf.Session() as session:
    session.run(initializer_op)
    
    for epoch in range(epochs):
        
        avg_cost = 0.0 # track cost to monitor performance during training
        
        # loop over all batches of the epoch:
        n_batches = int(mnist.train.num_examples/batch_size)
        for i in range(n_batches):
            
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            
            # feed batch data to run optimization and fetch cost: 
            _, c = session.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
            
            # compute average loss:
            avg_cost += c / n_batches
            
        # output logs at end of each epoch of training:
        print("Epoch:", '%03d' % (epoch+1), "/", epochs, "cost=", "{:.3f}".format(avg_cost))
    
    print("Training Complete. Testing Model.")
    
    # calculate accuracy by identifying test cases where the model's highest-probability class matches the true y label: 
    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch: 001 / 15 cost= 7.607
Epoch: 002 / 15 cost= 1.423
Epoch: 003 / 15 cost= 1.218
Epoch: 004 / 15 cost= 1.071
Epoch: 005 / 15 cost= 0.942
Epoch: 006 / 15 cost= 0.851
Epoch: 007 / 15 cost= 0.784
Epoch: 008 / 15 cost= 0.734
Epoch: 009 / 15 cost= 0.689
Epoch: 010 / 15 cost= 0.647
Epoch: 011 / 15 cost= 0.611
Epoch: 012 / 15 cost= 0.583
Epoch: 013 / 15 cost= 0.558
Epoch: 014 / 15 cost= 0.537
Epoch: 015 / 15 cost= 0.521
Training Complete. Testing Model.
Accuracy: 0.8532
